# Output Parser
**출력 파서(Output Parser)** 는 LLM의 출력값을 구조화된 형식으로 변환하고 답변에서 우리가 원하는 정보만 뽑아낼 때 유용하게 사용되는 도구다.
Application을 개발하다 보면 LLM의 답변을 우리가 원하는 형태로 받아야하는 경우가 있는데, 이 때 출력 파서를 이용하면 LLM의 응답을 구조화된 데이터로 받거나 원하는 정보를 손쉽게 추출하는 것이 가능하다(즉, 스키마 지원).

# PydanticOutputParser

`PydanticOutputParser`는 언어 모델의 출력을 **구조화된 정보**로 변환하는 데 도움을 주는 클래스입니다. 이 클래스는 단순 텍스트 응답 대신 **명확하고 체계적인 형태로 필요한 정보를 제공**할 수 있습니다.

이 클래스를 활용하면 언어 모델의 출력을 특정 데이터 모델에 맞게 변환하여 정보를 더 쉽게 처리하고 활용할 수 있습니다.

## 주요 메서드

`PydanticOutputParser` (대부분의 OutputParser에 해당)에는 주로 **두 가지 핵심 메서드**가 구현되어야 합니다.

- **`get_format_instructions()`**: 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침을 제공합니다. 예를 들어, 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환할 수 있습니다. 이 지침은 언어 모델이 출력을 구조화하고 특정 데이터 모델에 맞게 변환하는 데 매우 중요합니다.
- **`parse()`**: 언어 모델의 출력(문자열로 가정)을 받아 이를 특정 구조로 분석하고 변환합니다. Pydantic과 같은 도구를 사용하여 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환합니다.

## 참고 자료

- [Pydantic 공식 도큐먼트](https://docs.pydantic.dev/latest/)
- Pydantic은 파이썬에서 데이터 유효성을 검사하고, 데이터를 구조화된 형식으로 관리하는 데 유용한 라이브러리다.


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
# Bedrock 설정 및 Util 함수 설정
import sys
from langchain_aws import ChatBedrockConverse
import os
sys.path.append("C:/Users/younghl/CursorProjects/langchain-kr")
from Util.stream_utils import print_stream_content, get_stream_content

# 객체 생성
llm = ChatBedrockConverse(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    model="apac.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="ap-northeast-2",
)

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH03-OutputParser


다음은 이메일 본문 예시입니다.


In [5]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

출력 파서를 사용하지 않는 경우 예시

In [14]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

output = get_stream_content(answer)

In [13]:
print(output)

이메일의 중요한 내용을 정리하면 다음과 같습니다:

## 📧 이메일 주요 내용

**발신자**: 김철수 상무 (바이크코퍼레이션)  
**수신자**: 이은채 대리 (테디인터내셔널)

### 🎯 핵심 목적
- **ZENESIS 자전거** 유통 협력 제안

### 📋 주요 요청사항
1. **브로슈어 요청**
   - 기술 사양
   - 배터리 성능
   - 디자인 정보

2. **미팅 일정 제안**
   - **날짜**: 1월 15일 (화요일)
   - **시간**: 오전 10시
   - **장소**: 테디인터내셔널 사무실

### 🏢 회사 정보
- **바이크코퍼레이션**: 자전거 제조 및 유통 전문 기업
- **테디인터내셔널**: ZENESIS 자전거 출시 (최근 보도자료 발표)

### 📈 협력 범위
- 유통 전략 수립
- 마케팅 계획 구체화


위와 같은 이메일 내용이 주어졌을 때 아래의 Pydantic 스타일로 정의된 클래스를 사용하여 이메일의 정보를 파싱해 보겠습니다.

참고로, Field 안에 `description` 은 텍스트 형태의 답변에서 주요 정보를 추출하기 위한 설명입니다. LLM 이 바로 이 설명을 보고 필요한 정보를 추출하게 됩니다. 그러므로 이 설명은 정확하고 명확해야 합니다.

In [17]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [18]:
# instruction 을 출력합니다.
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


프롬프트를 정의합니다.

1. `question`: 유저의 질문을 받습니다.
2. `email_conversation`: 이메일 본문의 내용을 입력합니다.
3. `format`: 형식을 지정합니다.


In [19]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

다음으로는 Chain 을 생성합니다.


In [20]:
# chain 을 생성합니다.
chain = prompt | llm

체인을 실행하고 결과를 확인합니다.


In [21]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
output = get_stream_content(response)

마지막으로 `parser`를 사용하여 결과를 파싱하고 `EmailSummary` 객체로 변환합니다.


In [27]:
# PydanticOutputParser 를 사용하여 결과를 파싱합니다.
structured_output = parser.parse(output)
structured_output

EmailSummary(person='김철수 상무이사', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary="바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 신규 자전거 'ZENESIS' 모델에 대한 유통 협력을 제안하며, 상세 브로슈어(기술 사양, 배터리 성능, 디자인 정보)를 요청하고 유통 전략 및 마케팅 계획 논의를 위한 미팅을 제안함", date='1월 15일 화요일 오전 10시')

## parser 가 추가된 체인 생성

출력 결과를 정의한 Pydantic 객체로 생성할 수 있습니다.

In [25]:
# 출력 파서를 추가하여 전체 체인을 재구성합니다.
chain = prompt | llm | parser

In [30]:
# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력됩니다.
response

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary="바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 신규 자전거 'ZENESIS'에 대한 유통 협력을 제안하며, 제품 브로슈어(기술 사양, 배터리 성능, 디자인 정보)를 요청하고 협력 논의를 위한 미팅을 제안한 내용", date='1월 15일 화요일 오전 10시')

## with_structured_output()

`.with_structured_output(Pydantic)`을 사용하여 출력 파서를 추가하면, 출력을 Pydantic 객체로 변환할 수 있습니다.

참고로 이걸 쓰면, LLM이 좀더 구조화된 답변을 할 확률이 더 높아진다.

In [28]:
llm_with_structered = llm.with_structured_output(EmailSummary)

In [45]:
import logging

# invoke() 함수를 호출하여 결과를 출력합니다.
answer = llm_with_structered.invoke(email_conversation)
answer

INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
DEBUG:urllib3.connectionpool:https://bedrock-runtime.ap-northeast-2.amazonaws.com:443 "POST /model/apac.anthropic.claude-sonnet-4-20250514-v1%3A0/converse HTTP/1.1" 200 830


EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 신규 자전거 "ZENESIS"에 대한 유통 협력을 제안하며, 제품 브로슈어(기술 사양, 배터리 성능, 디자인 정보)를 요청하고 협력 논의를 위한 미팅을 제안함', date='1월 15일 오전 10시')

**참고**

한 가지 아쉬운 점은 `.with_structured_output()` 함수는 `stream()` 기능을 지원하지 않습니다.